In [4]:
from itertools import count
from collections import defaultdict


def printBoard(board, minute, blizzards, expedition):
    print(f"=== Minute {minute} ===")
    n, m = len(board), len(board[0])
    for i in range(n):
        for j in range(m):
            if (i, j) in blizzards:
                if len(blizzards[(i, j)]) == 1:
                    print(blizzards[(i, j)][0], end="")
                else:
                    print(len(blizzards[(i, j)]), end="")
            elif board[i][j] in "^v<>":
                print(".", end="")
            else:
                print(board[i][j], end="")
        print()
    print()


with open("24.input", "r") as f:
    blizzardsBoard = f.read().splitlines()
n, m = len(blizzardsBoard), len(blizzardsBoard[0])
expedition, exit = (0, 1), (n - 1, m - 2)
blizzards = defaultdict(
    list,
    {
        (i, j): blizzardsBoard[i][j]
        for i in range(n)
        for j in range(m)
        if blizzardsBoard[i][j] in "^v<>"
    },
)
# printBoard(blizzardsBoard, 0, blizzards, expedition)

seen = {str(blizzards): 0}
blizzardsHistory = [blizzards]
cycleLen = -1
cycleStart = -1

for minute in count(start=1):
    newBlizzards = defaultdict(list)
    for (i, j), blizzardList in blizzards.items():
        for d in blizzardList:
            if d == "^":
                nextPos = (i - 1 if i > 1 else n - 2, j)
                newBlizzards[nextPos].append("^")
            elif d == "v":
                nextPos = (i + 1 if i < n - 2 else 1, j)
                newBlizzards[nextPos].append("v")
            elif d == "<":
                nextPos = (i, j - 1 if j > 1 else m - 2)
                newBlizzards[nextPos].append("<")
            elif d == ">":
                nextPos = (i, j + 1 if j < m - 2 else 1)
                newBlizzards[nextPos].append(">")
    # printBoard(blizzardsBoard, minute, newBlizzards, expedition)
    if str(newBlizzards) in seen:
        print(minute, seen[str(newBlizzards)])
        cycleLen = minute - seen[str(newBlizzards)]
        cycleStart = seen[str(newBlizzards)]
        print(f"Cycle length: {cycleLen}", f"Cycle start: {cycleStart}", sep="\n")
        # printBoard(blizzardsBoard, minute, newBlizzards, expedition)
        break
    seen[str(newBlizzards)] = minute
    blizzardsHistory.append(newBlizzards)
    blizzards = newBlizzards


729 129
Cycle length: 600
Cycle start: 129


In [5]:
# BFS from the expedition
states = {(expedition, 0): 0}  # (pos, blizzardIndex) -> minute
expeditionPosCandidates = [expedition]
reachedExit = False
for minute in count(start=1):
    if reachedExit:
        break
    blizzardIndex = (
        minute if minute < cycleStart else cycleStart + (minute - cycleStart) % cycleLen
    )
    blizzards = blizzardsHistory[blizzardIndex]
    newExpeditionPosCandidates = []
    for pos in expeditionPosCandidates:
        if pos not in blizzards and (pos, blizzardIndex) not in states:
            # expedition can wait in place
            newExpeditionPosCandidates.append(pos)
            states[(pos, blizzardIndex)] = minute

        for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nextPos = (pos[0] + di, pos[1] + dj)
            if nextPos == exit:
                print(f"reached the exit in {minute} minutes")
                reachedExit = True
                break
            if nextPos == expedition or (
                0 < nextPos[0] < n - 1 and 0 < nextPos[1] < m - 1
            ):
                if nextPos not in blizzards and (nextPos, blizzardIndex) not in states:
                    newExpeditionPosCandidates.append(nextPos)
                    states[(nextPos, blizzardIndex)] = minute

        if reachedExit:
            break

    expeditionPosCandidates = newExpeditionPosCandidates


reached the exit in 308 minutes


In [6]:
def minuteToBlizzardIndex(minute):
    return (
        minute if minute < cycleStart else cycleStart + (minute - cycleStart) % cycleLen
    )


def bfs(startMinute, startPos, targetPos):
    states = set(
        [(startPos, minuteToBlizzardIndex(startMinute))]
    )  # (pos, blizzardIndex)
    positions = [startPos]
    for minute in count(start=startMinute + 1):
        blizzardIndex = minuteToBlizzardIndex(minute)
        blizzards = blizzardsHistory[blizzardIndex]
        newPositions = []
        for pos in positions:
            if pos not in blizzards and (pos, blizzardIndex) not in states:
                # expedition can wait in place
                newPositions.append(pos)
                states.add((pos, blizzardIndex))

            for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nextPos = (pos[0] + di, pos[1] + dj)
                if nextPos == targetPos:
                    print(
                        f"reached the target {targetPos} in {minute} minutes from {startPos}"
                    )
                    return minute
                if nextPos == startPos or (
                    0 < nextPos[0] < n - 1 and 0 < nextPos[1] < m - 1
                ):
                    if (
                        nextPos not in blizzards
                        and (nextPos, blizzardIndex) not in states
                    ):
                        newPositions.append(nextPos)
                        states.add((nextPos, blizzardIndex))
        positions = newPositions


t1 = bfs(0, expedition, exit)
t2 = bfs(t1, exit, expedition)
t3 = bfs(t2, expedition, exit)


reached the target (26, 120) in 308 minutes from (0, 1)
reached the target (0, 1) in 598 minutes from (26, 120)
reached the target (26, 120) in 908 minutes from (0, 1)
